In [1]:
# Import Required Libraries
import pandas as pd
import numpy as np
from faker import Faker
import os
import base64

fake = Faker()

In [2]:
# Create the Billing Table
FactTable = pd.read_csv("data/FactTable.csv", parse_dates=['dimDateServicePK', 'dimDatePostPK'], dayfirst=True)
DimHospital = pd.read_csv("data/DimHospital.csv")
Billing = pd.merge(FactTable, DimHospital, on='dimLocationPK', how='left')

# Apply Transformations
Billing["PaymentRecieved"] = Billing["Patient_Payment"] + Billing["AR"]
Billing["DiagnosisReportRef"] = "DIAG-" + Billing["FactTablePK"].astype(str)
Billing["ServiceReportRef"] = "SERV-" + Billing["FactTablePK"].astype(str)

column_mapping = {
    "FactTablePK" : "BillNbr",
    "dimDatePostPK" : "BillDate",
    "dimPatientPK" : "PatientNbr",
    "dimPhysicianPK" : "ProviderNbr",
    "LocationName" : "Facility",
    "Gross Expenses" : "BillAmount",
    "Insurance_Payment" : "InsuranceCoverage"
}

# Rename Columns
Billing = Billing.rename(columns=column_mapping)

# Select Columns
Billing = Billing[["BillNbr", "BillDate", "PatientNbr", "ProviderNbr", "Facility", "DiagnosisReportRef", "ServiceReportRef", "BillAmount", "InsuranceCoverage", "PaymentRecieved"]]

# Write the new Data to CSV File
Billing.to_csv('Spaghetti/Billing.csv', index=False)

C:\Users\Murtaza.h\AppData\Local\Temp\ipykernel_10332\326977405.py:2: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  FactTable = pd.read_csv("data/FactTable.csv", parse_dates=['dimDateServicePK', 'dimDatePostPK'], dayfirst=True)


In [3]:
# Create the Patient Table
DimPatient = pd.read_csv("data/DimPatient.csv")
MinServiceDate = FactTable.groupby("dimPatientPK").agg({"dimDateServicePK": "min"}).reset_index()
Patient = pd.merge(DimPatient, MinServiceDate, on='dimPatientPK', how='left')

# Apply Transformations
Patient.rename(columns={"Year of Birth": "year", "Month": "month", "Day": "day"}, inplace=True)
Patient["DOB"] = pd.to_datetime(Patient[["year", "month", "day"]])
Patient["RegistrationDate"] = Patient["dimDateServicePK"] - pd.to_timedelta(np.round(np.random.beta(0.5, 5, size=len(Patient)) * 10).astype(int), unit='days')
Patient["Address1"] = Patient.apply(lambda _: fake.secondary_address(), axis=1)
Patient["Address2"] = Patient.apply(lambda _: fake.building_number() + ' ' + fake.street_name(), axis=1)

column_mapping = {
    "dimPatientPK" : "PatientNbr",
    "PatientNumber" : "HealthCardNbr",
    "PatientGender" : "Gender",
    "Zip Codes" : "Zip"
}

# Rename Columns
Patient = Patient.rename(columns=column_mapping)

# Select Columns
Patient = Patient[["PatientNbr", "RegistrationDate", "HealthCardNbr", "FirstName", "LastName", "Email", "Gender", "DOB", "Address1", "Address2", "City", "State", "Zip"]]

# Write the new Data to CSV File
Patient.to_csv('Spaghetti/Patient.csv', index=False)

In [4]:
# Create the PatientSurvey Table
DimPatient = pd.read_csv("data/DimPatient.csv")
PatientSurvey = pd.merge(DimPatient, Patient, left_on='dimPatientPK', right_on='PatientNbr', how='left')

# Apply Transformations
PatientSurvey["SurveyDate"] = PatientSurvey["RegistrationDate"] + pd.to_timedelta(np.round(np.random.beta(1, 5, size=len(PatientSurvey)) * 3).astype(int), unit='days')
PatientSurvey["HeightIn"] = np.round(PatientSurvey["PatientHeight(in cms)"] / 2.54).astype(int)
PatientSurvey["WeightLbs"] = np.round(np.random.normal(150, 20, size=len(PatientSurvey)), 1)
PatientSurvey["WeightKgs"] = np.round(PatientSurvey["WeightLbs"] * 0.453592, 1)

column_mapping = {
    "PatientHeight(in cms)" : "HeightCms",
    "Tobacco" : "TobaccoUser",
    "Alcohol" : "AlcoholUser",
    "Exercise" : "ExerciseFrequency",
    "Diet" : "OnDiet"
}

# Rename Columns
PatientSurvey = PatientSurvey.rename(columns=column_mapping)

# Select Columns
PatientSurvey = PatientSurvey[["HealthCardNbr", "SurveyDate", "HeightCms", "HeightIn", "WeightLbs", "WeightKgs", "BloodGroup", "TobaccoUser", "AlcoholUser", "ExerciseFrequency", "OnDiet"]]

# Write the new Data to CSV File
PatientSurvey.to_csv('Spaghetti/PatientSurvey.csv', index=False)

In [5]:
# Create the Provider Table
DimPhysician = pd.read_csv("data/DimPhysician.csv")
DimSpeciality = pd.read_csv("data/DimSpeciality.csv")
Provider = pd.merge(DimPhysician, DimSpeciality, on='SpecialityCode', how='left')

# Apply Transformations
Provider["FirstName"] = Provider.apply(lambda _: fake.first_name(), axis=1)
Provider["Prefix"] = Provider["ProviderName"].apply(lambda name: 'Dr.' if 'Dr.' in name else '')
Provider["LastName"] = Provider["ProviderName"].apply(lambda name: name.replace('Dr. ', '') if 'Dr.' in name else name)
Provider["Email"] = Provider["FirstName"].str.lower() + '.' + Provider["LastName"].str.lower() + '@datacourse.com'

column_mapping = {
    "dimPhysicianPK" : "ProviderNbr",
    "ProviderNpi" : "NpiNbr",
    "ProviderFTE" : "FTE",
    "ProviderSpecialty" : "ProviderCategory"
}

# Rename Columns
Provider = Provider.rename(columns=column_mapping)

# Select Columns
Provider = Provider[["ProviderNbr", "NpiNbr", "FirstName", "LastName", "Email", "Prefix", "FTE", "ProviderCategory", "SpecialityCode"]]

# Write the new Data to CSV File
Provider.to_csv('Spaghetti/Provider.csv', index=False)

In [6]:
# Create the ProviderSpecialty Table
ProviderSpecialty = DimSpeciality.copy()

column_mapping = {
    "ProviderSpecialty" : "ProviderCategory",
    "SpecialityDesc" : "SpecialityDescription"
}

# Rename Columns
ProviderSpecialty = ProviderSpecialty.rename(columns=column_mapping)

# Select Columns
ProviderSpecialty = ProviderSpecialty[["SpecialityCode", "ProviderCategory", "SpecialityType", "SpecialityDescription"]]

# Write the new Data to CSV File
ProviderSpecialty.to_csv('Spaghetti/ProviderSpecialty.csv', index=False)

In [7]:
# Create the ClinicLocations Table
DimHospital = pd.read_csv("data/DimHospital.csv")
ClinicLocations = DimHospital.copy()

# Apply Transformations
ClinicLocations["Address1"] = ClinicLocations.apply(lambda _: fake.company() + ' Building', axis=1)
ClinicLocations["Address2"] = ClinicLocations.apply(lambda _: fake.building_number() + ' ' + fake.street_name(), axis=1)

column_mapping = {
    "dimLocationPK" : "ClinicNbr",
    "LocationName" : "DisplayName"
}

# Rename Columns
ClinicLocations = ClinicLocations.rename(columns=column_mapping)

# Select Columns
ClinicLocations = ClinicLocations[["ClinicNbr", "DisplayName", "Address1", "Address2", "City", "State", "Zip"]]

# Write the new Data to CSV File
ClinicLocations.to_csv('Spaghetti/ClinicLocations.csv', index=False)

In [8]:
# Create DiagnosisReport and ServiceReport Sub-tables
DimDiagnosisCode = pd.read_csv("data/DimDiagnosisCode.csv")
DimCptCode = pd.read_csv("data/DimCptCode.csv")
DiagnosisReport = pd.merge(FactTable, DimDiagnosisCode, on='dimDiagnosisCodePK', how='left')
ServiceReport = pd.merge(FactTable, DimCptCode, on='dimCPTCodePK', how='left')

# Apply Transformations for Diagnosis Report
DiagnosisReport["ReportNbr"] = DiagnosisReport["FactTablePK"]
DiagnosisReport["GeneratedDate"] = DiagnosisReport["dimDateServicePK"]
DiagnosisReport["ReportRefNbr"] = "DIAG-" + DiagnosisReport["FactTablePK"].astype(str)
DiagnosisReport["ReportType"] = 'Diagnosis'
DiagnosisReport["DiagnosisCode"] = DiagnosisReport["DiagnosisCode"]
DiagnosisReport["CPTUnits"] = np.NaN

ServiceReport["ReportNbr"] = ServiceReport["FactTablePK"] + 210964322
ServiceReport["GeneratedDate"] = ServiceReport["dimDatePostPK"]
ServiceReport["ReportRefNbr"] = "SERV-" + ServiceReport["FactTablePK"].astype(str)
ServiceReport["ReportType"] = 'Service'
ServiceReport["CPTCode"] = ServiceReport["CptCode"]

column_mapping = {
    "dimPatientPK" : "PatientNbr",
    "dimPhysicianPK" : "ApprovingDoctorNbr"
}

# Merge DiagnosisReport and ServiceReport
ClinicalReports = pd.concat([DiagnosisReport,ServiceReport],ignore_index=True)

# Rename Columns
ClinicalReports = ClinicalReports.rename(columns=column_mapping)

# Select Columns
ClinicalReports = ClinicalReports[["ReportNbr", "GeneratedDate", "PatientNbr", "ApprovingDoctorNbr", "ReportRefNbr", "ReportType", "CPTCode", "CPTUnits", "DiagnosisCode"]]

# Create a random PDF Binary for each row
ClinicalReports["ReportPDF"] = ClinicalReports.apply(lambda _: base64.b64encode(os.urandom(np.random.randint(100,300))).decode('utf-8'), axis=1)

# Write the new Data to CSV File
ClinicalReports.to_csv('Spaghetti/ClinicalReports.csv', index=False)

In [9]:
# Create CptCodesLookup Table
CptCodesLookup = DimCptCode[["CptCode", "CptGrouping", "CptDesc"]].drop_duplicates()

column_mapping = {
    "CptDesc" : "CptDescription"
}

# Rename Columns
CptCodesLookup = CptCodesLookup.rename(columns=column_mapping)

# Write the new Data to CSV File
CptCodesLookup.to_csv('Spaghetti/CptCodesLookup.csv', index=False)

In [10]:
# Create DiagnosisCodeLookup Table
DiagnosisCodeLookup = DimDiagnosisCode[["DiagnosisCode", "DiagnosisCodeGroup", "DiagnosisCodeDescription"]].drop_duplicates()

column_mapping = {
    "DiagnosisCodeGroup" : "DiagnosisGrouping",
    "DiagnosisCodeDescription" : "DiagnosisDescription"
}

# Rename Columns
DiagnosisCodeLookup = DiagnosisCodeLookup.rename(columns=column_mapping)

# Write the new Data to CSV File
DiagnosisCodeLookup.to_csv('Spaghetti/DiagnosisCodeLookup.csv', index=False)